In [1]:
!pip install -q PyMuPDF
!pip install -q mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 57.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Extract Links

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the webpage
url = "https://transparencia.oviedo.es/normativa-e-informes/ordenanzas-y-reglamentos"
# Define headers to mimic a web browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

# Make a request to the webpage with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the ul element with the class 'layouts level-1'
ul_element = soup.find('ul', class_='layouts level-1')

# Find all li elements within the ul element
li_elements = ul_element.find_all('li')

# Extract the href attributes and span text from each li element
result = []
for li in li_elements:
    a_tag = li.find('a')
    href = a_tag['href']
    span_text = a_tag.find('span').text
    # Check if the last segment of the URL contains the desired words
    last_segment = href.split('/')[-1]
    if any(keyword in last_segment for keyword in ['reglamentos', 'ordenanza', 'ordenanzas']):
        result.append((span_text, href))


# # Create a DataFrame
df_links = pd.DataFrame(result, columns=['subgrupo', "link_grupo"])

df_links

,subgrupo,link_grupo
0,Ordenanzas reguladoras,https://transparencia.oviedo.es/normativa-e-in...
1,Reglamentos,https://transparencia.oviedo.es/normativa-e-in...
2,Ordenanza de subvenciones y bases,https://transparencia.oviedo.es/normativa-e-in...
3,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/ordenanzas-de-...


# Primer Subgrupo: Ordenanzas reguladoras

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import fitz  # PyMuPDF
import io
from datetime import datetime

# URL of the webpage
url = df_links.link_grupo[0]

# Define headers to mimic a web browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

# Make a request to the webpage with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the data
data = []


# Function to extract PDF content from URL using PyMuPDF
def extract_pdf_content(pdf_url):
    response = requests.get(pdf_url, headers=headers)
    if response.headers.get('Content-Type') == 'application/pdf':
        pdf_file = io.BytesIO(response.content)
        document = fitz.open(stream=pdf_file, filetype="pdf")
        text_content = ""
        for page_num in range(len(document)):
            page = document.load_page(page_num)
            text_content += page.get_text()
        return text_content
    else:
        return "The URL did not return a PDF file."



# Find all h3 elements
h3_elements = soup.find_all('h3', class_='asset-entries-group-label')
for h3 in h3_elements:
    h3_text = h3.text.strip()

    # Find the sibling div elements with class 'assets-default'
    sibling_div = h3.find_next_sibling('div', class_='assets-default')
    if sibling_div:
        # Find all h4 elements and a href within the 'text' div
        h4_elements = sibling_div.find_all('h4', class_='pagesubsubtitle')
        for h4 in h4_elements:
            h4_text = h4.text.strip()
            a_tags = h4.find_next_sibling('p').find_all('a')
            for a in a_tags:
                href = a['href']

                # Add prefix if href starts with "/documents"
                if href.startswith('/documents'):
                    href = "https://transparencia.oviedo.es" + href

                content = extract_pdf_content(href)

                data.append((h4_text, h3_text, href, content))

# Create a DataFrame
df_org_reg = pd.DataFrame(data, columns=['titulo', 'grupo', 'url', 'content'])

ciudad = "Oviedo"
current_date = datetime.today().strftime('%Y-%m-%d')
subgroup = df_links.subgrupo[0]

df_org_reg.insert(0, 'ciudad', ciudad)
df_org_reg.insert(1, 'date', current_date)
df_org_reg.insert(4, 'subgrupo', subgroup)


# Save or display the DataFrame
df_org_reg

,ciudad,date,titulo,grupo,subgrupo,url,content
0,Oviedo,2024-07-28,Ordenanza municipal de Limpieza de vías públic...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/OR...,\n \nOrdenanza municipal de Limpieza de Vías ...
1,Oviedo,2024-07-28,Ordenanza de protección del medio ambiente atm...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,ORDENANZA MUNICIPAL DE PROTECCIÓN DEL \nMEDIO ...
2,Oviedo,2024-07-28,Ordenanza sobre protección del medio ambiente ...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,\t\n \r\r \...
3,Oviedo,2024-07-28,Ordenanza municipal de convivencia ciudadana,Seguridad Ciudadana,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,1\nTEXTO DE LA ORDENANZA MUNICIPAL DE CONVIVEN...
4,Oviedo,2024-07-28,Ordenanza municipal de movilidad y tráfico,Seguridad Ciudadana,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,AYUNTAMIENTO DE OVIEDO CIF: P3304400I Registr...
5,Oviedo,2024-07-28,"Ordenanza municipal de transparencia, acceso a...",Participación,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/OR...,"\n1 \nORDENANZA MUNICIPAL DE TRANSPARENCIA, ..."
6,Oviedo,2024-07-28,Ordenanza municipal reguladora de la cesión te...,Participación,Ordenanzas reguladoras,https://transparencia.oviedo.es/documents/2504...,DOCUMENTO\nIDENTIFICADORES\nESTADO\nFIRMAS\nOT...
7,Oviedo,2024-07-28,Ordenanza reguladora de la administración elec...,Participación,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,TEXTO INTEGRO DE LA ORDENANZA REGULADORA DE LA...
8,Oviedo,2024-07-28,Ordenanza reguladora de los Servicios Locales ...,Participación,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,\t\n \r \r\r\r...
9,Oviedo,2024-07-28,Ordenanza municipal reguladora de las ocupacio...,Uso de vías y espacios públicos. Obras,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/OR...,1 \n \n \n \n \nORDENANZA MUNICIPAL REGULADORA...


In [5]:
df_org_reg.to_csv('/content/drive/MyDrive/Colab Notebooks/14 Scraping/Asturias_Oviedo/oviedo_ordenanzas_reguladoras.csv', index=False)

In [6]:
# pd.read_csv('/content/drive/MyDrive/Colab Notebooks/14 Scraping/Asturias_Oviedo/oviedo_ordenanzas_reguladoras.csv', index_col = False)

# Segundo Subgrupo: Ordenanza de subvenciones y bases

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import fitz  # PyMuPDF
import io

# Define the URL
url = df_links.link_grupo[2]

# Define headers to mimic a web browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

# Make a request to the webpage with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Initialize lists to store the extracted data
data = []

# Function to extract PDF content from URL using PyMuPDF
def extract_pdf_content(pdf_url):
    try:
        response = requests.get(pdf_url, headers=headers, timeout=10)
        response.raise_for_status()
        if response.headers.get('Content-Type') == 'application/pdf':
            pdf_file = io.BytesIO(response.content)
            document = fitz.open(stream=pdf_file, filetype="pdf")
            text_content = ""
            for page_num in range(len(document)):
                page = document.load_page(page_num)
                text_content += page.get_text()
            return text_content
        else:
            return "The URL did not return a PDF file."
    except requests.exceptions.RequestException as e:
        return f"Failed to retrieve PDF content: {e}"

# Find all <h3> elements with class "asset-entries-group-label"
h3_elements = soup.find_all('h3', class_='asset-entries-group-label')

for h3 in h3_elements:
    # Get the text of the <h3> element
    h3_text = h3.get_text(strip=True)

    # Get the next sibling elements of <h3> to find the relevant <p> and <a> elements
    sibling = h3.find_next_sibling('div', class_='assets-default')

    if sibling:
        # Find all <p> elements within the sibling div
        p_elements = sibling.find_all('p')


        # Loop through each <p> element and find <strong> tags
        for p in p_elements:
            strong_p = p.find('strong')
            if strong_p:
                p_text = strong_p.get_text(strip=True)

            a_element = p.find('a', href=True)
            if a_element:
                href = a_element['href']

                # Append the extracted data to the list as a dictionary
                data.append({
                    'titulo': p_text,
                    'grupo': h3_text,
                    'url': href,
                    'content': content

                })

# Create a DataFrame
df_org_sub = pd.DataFrame(data, columns=['titulo', 'grupo', 'url', 'content'])

ciudad = "Oviedo"
current_date = datetime.today().strftime('%Y-%m-%d')
subgroup = df_links.subgrupo[2]

df_org_sub.insert(0, 'ciudad', ciudad)
df_org_sub.insert(1, 'date', current_date)
df_org_sub.insert(4, 'subgrupo', subgroup)


# Save or display the DataFrame
df_org_sub

,ciudad,date,titulo,grupo,subgrupo,url,content
0,Oviedo,2024-07-28,Subvenciones a entidades organizadoras de fies...,Festejos,Ordenanza de subvenciones y bases,https://www.oviedo.es/documents/25047/1519956/...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...
1,Oviedo,2024-07-28,Subvenciones destinadas a clubes deportivos de...,Deportes,Ordenanza de subvenciones y bases,https://sede.oviedo.es/documents/25047/1520881...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...
2,Oviedo,2024-07-28,Subvenciones destinadas a jóvenes deportistas ...,Deportes,Ordenanza de subvenciones y bases,https://sede.oviedo.es/documents/25047/1520881...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...
3,Oviedo,2024-07-28,Subvenciones destinadas a clubes deportivos y ...,Deportes,Ordenanza de subvenciones y bases,https://sede.oviedo.es/documents/25047/1520881...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...
4,Oviedo,2024-07-28,"Subvenciones destinadas a escuelas deportivas,...",Deportes,Ordenanza de subvenciones y bases,https://sede.oviedo.es/documents/25047/1520881...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...
5,Oviedo,2024-07-28,Subvenciones para entidades sin ánimo de lucro...,Servicios Sociales,Ordenanza de subvenciones y bases,https://sede.oviedo.es/documents/25047/1520877...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...
6,Oviedo,2024-07-28,Subvenciones para proyectos para el fomento de...,Servicios Sociales,Ordenanza de subvenciones y bases,https://sede.oviedo.es/documents/25047/1520877...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...
7,Oviedo,2024-07-28,Subvenciones para Organizaciones no Gubernamen...,Servicios Sociales,Ordenanza de subvenciones y bases,https://sede.oviedo.es/documents/25047/1520877...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...
8,Oviedo,2024-07-28,Ayudas a familias en dificultades económicas c...,Servicios Sociales,Ordenanza de subvenciones y bases,https://sede.oviedo.es/documents/25047/1520877...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...
9,Oviedo,2024-07-28,Prestaciones sociales de carácter económico pa...,Servicios Sociales,Ordenanza de subvenciones y bases,https://sede.oviedo.es/documents/25047/1520877...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...


In [8]:
df_org_sub.to_csv('/content/drive/MyDrive/Colab Notebooks/14 Scraping/Asturias_Oviedo/oviedo_ordenanzas_subvenciones.csv', index=False)

# Tercer Subgrupo: Ordenanzas de tributos y precios públicos

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
url = df_links.link_grupo[3]

# Define headers to mimic a web browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

# Make a request to the webpage with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Initialize lists to store the extracted data
data = []


# Function to extract PDF content from URL using PyMuPDF
def extract_pdf_content(pdf_url):
    try:
        response = requests.get(pdf_url, headers=headers, timeout=10)
        response.raise_for_status()
        if response.headers.get('Content-Type') == 'application/pdf':
            pdf_file = io.BytesIO(response.content)
            document = fitz.open(stream=pdf_file, filetype="pdf")
            text_content = ""
            for page_num in range(len(document)):
                page = document.load_page(page_num)
                text_content += page.get_text()
            return text_content
        else:
            return "The URL did not return a PDF file."
    except requests.exceptions.RequestException as e:
        return f"Failed to retrieve PDF content: {e}"


# Find all <h3> elements with class "asset-entries-group-label"
div_elements = soup.find_all('div', class_='portlet-body')
# div_elements

# Iterate over each div element
for div in div_elements:
    # Extract the portlet title if it exists
    portlet_title = div.find('div', class_='portlet-title')

    if portlet_title:
        portlet_title_text = portlet_title.get_text(strip=True)

        # print(portlet_title_text)


    # Extract all 'li' elements with class 'document-entry'
    li_elements = div.find_all('li', class_='document-entry')
    for li in li_elements:
        a_tag = li.find('a')
        if a_tag:
            href = a_tag['href']
            # Add prefix if href starts with "/documents"
            if href.startswith('/documents'):
              href = "https://transparencia.oviedo.es" + href

            content = extract_pdf_content(href)

            link_text = a_tag.get_text(strip=True)

            # Add the extracted information to the data list
            data.append((link_text, portlet_title_text ,href, content))


# Create a DataFrame
df_org_trib = pd.DataFrame(data, columns=['titulo', 'grupo', 'url', 'content'])

ciudad = "Oviedo"
current_date = datetime.today().strftime('%Y-%m-%d')
subgroup = df_links.subgrupo[3]

df_org_trib.insert(0, 'ciudad', ciudad)
df_org_trib.insert(1, 'date', current_date)
df_org_trib.insert(4, 'subgrupo', subgroup)


# Save or display the DataFrame
df_org_trib

,ciudad,date,titulo,grupo,subgrupo,url,content
0,Oviedo,2024-07-28,"Ordenanza General(pdf781,2 kB )","Ordenanza General de Gestión, Recaudación e In...",Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,"\n1 \n \n \nORDENANZA GENERAL DE GESTIÓN, REC..."
1,Oviedo,2024-07-28,Ordenanza 100. Tasa por expedición de document...,Por prestación de servicios o realización de a...,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,\n1 \n \nORDENANZA FISCAL NÚMERO 100 \nTASA P...
2,Oviedo,2024-07-28,Ordenanza 101. Licencia de autotaxis y demás v...,Por prestación de servicios o realización de a...,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,\n1 \n \n \n \nORDENANZA FISCAL NÚMERO 101 \n...
3,Oviedo,2024-07-28,Ordenanza 102. Servicios especiales por espect...,Por prestación de servicios o realización de a...,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,\n1 \n \n \nORDENANZA FISCAL NÚMERO 102 \n \n...
4,Oviedo,2024-07-28,Ordenanza 103. Tasa por licencias urbanísticas...,Por prestación de servicios o realización de a...,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,\n1 \n \n \n \nORDENANZA FISCAL NÚMERO 103 \n...
5,Oviedo,2024-07-28,Ordenanza 104. Tasa de procedimientos de inter...,Por prestación de servicios o realización de a...,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,\n1 \n \n \nORDENANZA FISCAL NÚMERO 104 \n \n...
6,Oviedo,2024-07-28,Ordenanza 105. Tasa por el mantenimiento y pre...,Por prestación de servicios o realización de a...,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,\n1 \n \n \nORDENANZA FISCAL NÚMERO 105 \n \n...
7,Oviedo,2024-07-28,Ordenanza 108. Tasa por recogida de basuras.(p...,Por prestación de servicios o realización de a...,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,\n1 \n \n \nORDENANZA FISCAL NÚMERO 108 \nTAS...
8,Oviedo,2024-07-28,"Ordenanza 110. Inmovilización, recogida o depó...",Por prestación de servicios o realización de a...,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,\n1 \n \n \nORDENANZA FISCAL NUMERO 110 \n \n...
9,Oviedo,2024-07-28,Ordenanza 111. Vigilancia Establecimientos(pdf...,Por prestación de servicios o realización de a...,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,\n1 \n \n \nORDENANZA FISCAL NUMERO 111 \n \n...


In [10]:
df_org_trib.to_csv('/content/drive/MyDrive/Colab Notebooks/14 Scraping/Asturias_Oviedo/oviedo_ordenanzas_tributos.csv', index=False)

# Concatenate DFs

In [11]:
df_org_reg.head(2)

,ciudad,date,titulo,grupo,subgrupo,url,content
0,Oviedo,2024-07-28,Ordenanza municipal de Limpieza de vías públic...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/OR...,\n \nOrdenanza municipal de Limpieza de Vías ...
1,Oviedo,2024-07-28,Ordenanza de protección del medio ambiente atm...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,ORDENANZA MUNICIPAL DE PROTECCIÓN DEL \nMEDIO ...


In [12]:
df_org_sub.head(2)

,ciudad,date,titulo,grupo,subgrupo,url,content
0,Oviedo,2024-07-28,Subvenciones a entidades organizadoras de fies...,Festejos,Ordenanza de subvenciones y bases,https://www.oviedo.es/documents/25047/1519956/...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...
1,Oviedo,2024-07-28,Subvenciones destinadas a clubes deportivos de...,Deportes,Ordenanza de subvenciones y bases,https://sede.oviedo.es/documents/25047/1520881...,ORDENANZA MUNICIPAL SOBRE EVALUACIÓN DE RIESGO...


In [13]:
df_org_trib.head(2)

,ciudad,date,titulo,grupo,subgrupo,url,content
0,Oviedo,2024-07-28,"Ordenanza General(pdf781,2 kB )","Ordenanza General de Gestión, Recaudación e In...",Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,"\n1 \n \n \nORDENANZA GENERAL DE GESTIÓN, REC..."
1,Oviedo,2024-07-28,Ordenanza 100. Tasa por expedición de document...,Por prestación de servicios o realización de a...,Ordenanzas de tributos y precios públicos.,https://transparencia.oviedo.es/documents/2504...,\n1 \n \nORDENANZA FISCAL NÚMERO 100 \nTASA P...


In [14]:
final_df_oviedo = pd.concat([df_org_reg, df_org_sub, df_org_trib], axis=0, ignore_index=True)

In [15]:
final_df_oviedo.head()

,ciudad,date,titulo,grupo,subgrupo,url,content
0,Oviedo,2024-07-28,Ordenanza municipal de Limpieza de vías públic...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/OR...,\n \nOrdenanza municipal de Limpieza de Vías ...
1,Oviedo,2024-07-28,Ordenanza de protección del medio ambiente atm...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,ORDENANZA MUNICIPAL DE PROTECCIÓN DEL \nMEDIO ...
2,Oviedo,2024-07-28,Ordenanza sobre protección del medio ambiente ...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,\t\n \r\r \...
3,Oviedo,2024-07-28,Ordenanza municipal de convivencia ciudadana,Seguridad Ciudadana,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,1\nTEXTO DE LA ORDENANZA MUNICIPAL DE CONVIVEN...
4,Oviedo,2024-07-28,Ordenanza municipal de movilidad y tráfico,Seguridad Ciudadana,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,AYUNTAMIENTO DE OVIEDO CIF: P3304400I Registr...


# Create SQL

In [16]:
!apt-get update
!apt-get install -y mysql-server mysql-client
!service mysql start
!mysql -e "CREATE DATABASE testdb;"
!mysql -e "CREATE USER 'testuser'@'localhost' IDENTIFIED BY 'testpassword';"
!mysql -e "GRANT ALL PRIVILEGES ON testdb.* TO 'testuser'@'localhost';"
!mysql -e "GRANT PROCESS, RELOAD, SHOW DATABASES ON *.* TO 'testuser'@'localhost';"
!mysql -e "FLUSH PRIVILEGES;"

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,129 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,206 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://archi

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import BytesIO
import mysql.connector

# Configuración de la conexión a MySQL
mydb = mysql.connector.connect(
    host="localhost",
    user="testuser",
    password="testpassword",
    database="testdb"
)

mycursor = mydb.cursor()


# Create the table in the database if it doesn't exist and adjust column types
create_table_query = """
CREATE TABLE IF NOT EXISTS normativa (
    id INT AUTO_INCREMENT PRIMARY KEY,
    ciudad VARCHAR(255),
    date DATE,
    titulo TEXT,
    grupo VARCHAR(255),
    subgrupo TEXT,
    url TEXT,
    content MEDIUMTEXT
)
"""

mycursor.execute(create_table_query)

# Insertar los datos en la tabla de MySQL
try:
  for index, row in final_df_oviedo.iterrows():
      sql = "INSERT INTO normativa (ciudad, date, titulo, grupo, subgrupo, url, content) VALUES (%s, %s, %s, %s, %s, %s, %s)"
      val = (row['ciudad'], row['date'], row['titulo'], row['grupo'], row['subgrupo'], row['url'], row['content'])

      mycursor.execute(sql, val)
      print(row['titulo'], " insertado en tabla.")

  mydb.commit()
  print(mycursor.rowcount, "registro(s) insertado(s).")

except mysql.connector.Error as err:
  print("Error: {}".format(err))
# finally:
#     # Close the cursor and connection
#     mycursor.close()
#     mydb.close()

Ordenanza municipal de Limpieza de vías públicas y recogida de residuos domésticos  insertado en tabla.
Ordenanza de protección del medio ambiente atmosférico  insertado en tabla.
Ordenanza sobre protección del medio ambiente contra la emisión de ruidos y vibraciones  insertado en tabla.
Ordenanza municipal de convivencia ciudadana  insertado en tabla.
Ordenanza municipal de movilidad y tráfico  insertado en tabla.
Ordenanza municipal de transparencia, acceso a la información y reutilización  insertado en tabla.
Ordenanza municipal reguladora de la cesión temporal de uso de locales a entidades sin ánimo de lucro  insertado en tabla.
Ordenanza reguladora de la administración electrónica  insertado en tabla.
Ordenanza reguladora de los Servicios Locales de Consumo  insertado en tabla.
Ordenanza municipal reguladora de las ocupaciones de espacios públicos  insertado en tabla.
Ordenanza Reguladora de actividades comerciales e industriales en el espacio de dominio y uso público del municipi

In [18]:
# Export the database to a SQL dump file
!mysqldump -u testuser -ptestpassword testdb > "/content/drive/MyDrive/Colab Notebooks/14 Scraping/Asturias_Oviedo/oviedo.sql"

mysqldump: [Warning] Using a password on the command line interface can be insecure.


In [19]:
query = "SELECT * FROM normativa"
df = pd.read_sql(query, mydb)

# Display the DataFrame
df.head()

<ipython-input-19-dac1cc8ec922>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, mydb)


,id,ciudad,date,titulo,grupo,subgrupo,url,content
0,1,Oviedo,2024-07-28,Ordenanza municipal de Limpieza de vías públic...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/OR...,\n \nOrdenanza municipal de Limpieza de Vías ...
1,2,Oviedo,2024-07-28,Ordenanza de protección del medio ambiente atm...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,ORDENANZA MUNICIPAL DE PROTECCIÓN DEL \nMEDIO ...
2,3,Oviedo,2024-07-28,Ordenanza sobre protección del medio ambiente ...,Medio Ambiente,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,\t\n \r\r \...
3,4,Oviedo,2024-07-28,Ordenanza municipal de convivencia ciudadana,Seguridad Ciudadana,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,1\nTEXTO DE LA ORDENANZA MUNICIPAL DE CONVIVEN...
4,5,Oviedo,2024-07-28,Ordenanza municipal de movilidad y tráfico,Seguridad Ciudadana,Ordenanzas reguladoras,https://www.oviedo.es/documents/25047/25102/Or...,AYUNTAMIENTO DE OVIEDO CIF: P3304400I Registr...


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        81 non-null     int64 
 1   ciudad    81 non-null     object
 2   date      81 non-null     object
 3   titulo    81 non-null     object
 4   grupo     81 non-null     object
 5   subgrupo  81 non-null     object
 6   url       81 non-null     object
 7   content   81 non-null     object
dtypes: int64(1), object(7)
memory usage: 5.2+ KB


In [22]:
import pandas as pd
from sqlalchemy import create_engine

# Define database connection parameters
db_config = {
    "host":"localhost",
    "user":"testuser",
    "password":"testpassword",
    "database":"testdb"
}

# Establish the connection
connection_url = f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}"

# Create an SQLAlchemy engine
engine = create_engine(connection_url)

# Define the SQL query to select data
query = "SELECT * FROM normativa"

# Load data into a DataFrame
df = pd.read_sql(query, engine)

# Print the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        81 non-null     int64 
 1   ciudad    81 non-null     object
 2   date      81 non-null     object
 3   titulo    81 non-null     object
 4   grupo     81 non-null     object
 5   subgrupo  81 non-null     object
 6   url       81 non-null     object
 7   content   81 non-null     object
dtypes: int64(1), object(7)
memory usage: 5.2+ KB


# Cuarto Subgrupo: Reglamentos

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL
url = df_links.link_grupo[1]

# Define headers to mimic a web browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

# Make a request to the webpage with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Initialize lists to store the extracted data
data = []

# Find all h3 elements
h3_elements = soup.find_all('h3', class_='asset-entries-group-label')

# Iterate over each h3 element to extract the required information
for h3 in h3_elements:
    h3_text = h3.text.strip()

    # Find the sibling div elements with class 'box'
    sibling_div = h3.find_next_sibling('div', class_='box')

    # If the sibling div exists, find all a tags within it
    if sibling_div:
        a_tags = sibling_div.find_all('a', class_='link-entry')
        for a_tag in a_tags:
            href = a_tag['href']
            link_text = a_tag.get_text(strip=True)
            # Add the extracted information to the data list
            data.append((link_text, href, h3_text))

# Convert the data to a DataFrame
df_reg = pd.DataFrame(data, columns=['titulo', 'url_reglamentos', 'grupo'])

# Function to check for TEXTO CONSOLIDADO in each URL
def check_texto_consolidado(url):
    try:
        # Make a request to the URL
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Ensure the request was successful

        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all <p> elements
        p_elements = soup.find_all('p')

        # Loop through each <p> element to find <strong> with text "TEXTO CONSOLIDADO"
        for p in p_elements:
            strong_p = p.find('strong')
            if strong_p and "TEXTO CONSOLIDADO" in strong_p.text:
                a_tag = p.find('a')
                if a_tag:
                    return a_tag['href']
        return ""
    except requests.RequestException:
        return ""

# Function to extract PDF content from URL using PyMuPDF
def extract_pdf_content(pdf_url):
    try:
        response = requests.get(pdf_url, headers=headers, timeout=10)
        response.raise_for_status()
        if response.headers.get('Content-Type') == 'application/pdf':
            pdf_file = io.BytesIO(response.content)
            document = fitz.open(stream=pdf_file, filetype="pdf")
            text_content = ""
            for page_num in range(len(document)):
                page = document.load_page(page_num)
                text_content += page.get_text()
            return text_content
        else:
            return "The URL did not return a PDF file."
    except requests.exceptions.RequestException as e:
        return f"Failed to retrieve PDF content: {e}"

# Apply the function to each URL in the DataFrame
df_reg['url'] = df_reg['url_reglamentos'].apply(check_texto_consolidado)

# Extract PDF content for each Texto Consolidado URL
df_reg['content'] = df_reg['url'].apply(lambda x: extract_pdf_content(x) if x else "")

ciudad = "Oviedo"
current_date = datetime.today().strftime('%Y-%m-%d')
subgroup = df_links.subgrupo[3]

df_reg.drop(['url_reglamentos'], axis=1, inplace=True)

df_reg.insert(0, 'ciudad', ciudad)
df_reg.insert(1, 'date', current_date)
df_reg.insert(4, 'subgrupo', subgroup)


# Save or display the DataFrame
df_reg